# Sequential Agent

If you have a flow you need built. Railtracks can do it very simply

The magic of railtracks is everything can be done in code, there is no API you need to learn, its pure python logic

In [2]:

import railtracks as rt
from dotenv import load_dotenv
from tavily import TavilyClient
import os

_ = load_dotenv()

tav_client = TavilyClient(os.environ["TAVILY_SEARCH_API_KEY"])

## Lead Pipeline

Image to come.

In [ ]:
from collections import defaultdict

def format_result(title: str, content: str, url: str) -> str:
    return f"Title: {title}\nContent: {content}\nURL: {url}\n"

@rt.function_node
def query_internet(query: str) -> str:
    results = tav_client.search(query, num_results=5)


    formatted_results = "\n".join([format_result(res.get('title', 'Title Unavailable'), res.get('content', 'Content Unavailable'), res.get('url', "Unknown")) for res in results["results"]])
    return f"Top search results for '{query}':\n\n{formatted_results}"



def format_thoughts(thoughts: dict[str, str | None]):
    strings = []
    for thought, resolved in thoughts.items():
        strings.append(f"- {thought}: {resolved if resolved is not None else 'Unresolved'}")
    return "\n".join(strings)


@rt.function_node
def add_thought(thought: str) -> str:
    current_phase = rt.context.get("current_phase")

    thought_store = rt.context.get("thought_store")

    thought_store[current_phase][thought] = None

    task_list_str = format_thoughts(thought_store[current_phase])

    return f"Added thought {thought}.\n\nThe current thought list contains\n{task_list_str}\n"

@rt.function_node
def review_thoughts():
    current_phase = rt.context.get("current_phase")

    thought_store = rt.context.get("thought_store")

    return format_thoughts(thought_store[current_phase])

@rt.function_node
def resolve_thoughts(thought: str, resolved: str):
    """Resolves a given thought with the text provided
    """

    thought_store = rt.context.get("thought_store")
    current_phase = rt.context.get("current_phase")
    if thought not in thought_store[current_phase]:
        raise KeyError(f"Thought '{thought}' does not exist.")
    
    thought_store[current_phase][thought] = resolved

    return f"Resolved {thought} with {resolved}"


thought_manager = [resolve_thoughts, review_thoughts, add_thought]




Top search results for 'Tell me about google':

Title: 12-35 Tell me about Google Technology - Google 101 Facts
Content: Google received high marks for its accurate and relevant search results, as well as its user-friendly interface and wide range of features. However, each search
URL: https://www.steegle.com/about/google-101-facts/12-35-google-technology

Title: What is Google? - Tech Monitor
Content: # What is Google? The company has operated since the late 1990s and is one of the most fundamental forces in the development of the internet as we know it – to the point where the phrase ‘googling’ is synonymous with using a search engine. While the search engine is the company’s flagship product, Google operates in numerous areas including hardware, cloud computing, advertising, software and AI. Google is now under the umbrella of Alphabet Inc., a publicly traded company with multiple classes of shareholders. Like many of the major public-facing tech companies, advertising is at the hea

In [31]:
from typing import Optional, List
from pydantic import BaseModel, Field

class Companies(BaseModel):
   companies: List["Company"] = Field(description="List of qualified companies")
class Company(BaseModel):
    name: str = Field(description="Official company name")
    employee_count: Optional[int] = Field(description="Number of employees")
    estimated_revenue: Optional[str] = Field(description="Estimated annual revenue")
    area_of_expertise: str = Field(description="Primary business focus or industry vertical")
    tech_stack: List[str] = Field(default_factory=list, description="Known technologies and tools in use")
    url: str = Field(description="Company website URL")
    linkedin: str = Field(description="Company LinkedIn profile URL")
    why_good_fit: str = Field(description="Brief explanation of why this company is a good prospect for the product offering")
    data_sources: List[str] = Field(default_factory=list, description="URLs where information was gathered")


LEAD_FINDER_SYSTEM = """You are an in-depth research agent specialized in identifying and qualifying companies as potential sales prospects.

# YOUR MISSION
You will be provided with a product offering or service to sell. Your job is to find 10-20 companies that would be excellent prospects for this offering. You are the FIRST FILTER in a larger sales process - your goal is to identify and lightly qualify companies, not to do exhaustive research on each one.

# RESEARCH METHODOLOGY

## Phase 1: Strategic Planning (Use Thought Management)
Before searching, you MUST use the thought management tools to plan your approach:
- Use `add_thought` to brainstorm: What industries would need this product? What company characteristics indicate a good fit?
- Document your search strategy: What search queries will you run? What signals will you look for?
- Create a research checklist of information you need to gather for each company

## Phase 2: Broad Discovery
- Execute broad searches to identify potential companies in relevant industries
- Cast a wide net - look for companies that MIGHT fit based on industry, size, or business model
- Use `add_thought` to track promising leads and patterns you notice
- Use `resolve_thoughts` to mark completed searches and document findings

## Phase 3: Company Qualification (Multi-Pass)
For each promising company, conduct a focused investigation:

**First Pass - Basic Fit:**
- Confirm the company is real and active
- Verify industry alignment with the product offering
- Check approximate company size (employees)
- Initial assessment: Does this seem like a prospect worth investigating further?

**Second Pass - Deeper Details:**
- Find specific employee count and revenue estimates
- Identify their current tech stack or tools (look at job postings, tech detection sites, engineering blogs, case studies)
- Find their LinkedIn company page
- Determine WHY they would need your product offering

**Between passes:** Use `review_thoughts` regularly to ensure you're staying on track and haven't missed anything.

## Phase 4: Documentation & Finalization
- Use `resolve_thoughts` to mark each company as fully researched
- Ensure you have 10-20 high-confidence companies
- Double-check that each company has all required fields, especially LinkedIn URLs

# CRITICAL REQUIREMENTS

1. **Always use thought management tools BEFORE conducting research**
   - Never start searching without first planning your approach
   - Regularly check in with your thought list to track progress

2. **Only include HIGH CONFIDENCE companies**
   - You must be confident in the fit between the company and the product
   - If you can't find sufficient information, move on to other prospects

3. **Maximum 20 companies**
   - Focus on quality over quantity
   - Better to have 12 excellent prospects than 20 mediocre ones

4. **LinkedIn is mandatory**
   - Every company MUST have a LinkedIn URL
   - This is critical for downstream processes

5. **Think about the pitch angle**
   - For each company, you should have a clear sense of why they need this product
   - This helps you evaluate fit and will be documented in `why_good_fit`

# SEARCH STRATEGY TIPS

- Use multiple search queries per industry vertical
- Look for recent news, funding announcements, or growth signals
- Check job postings to understand their tech stack and pain points
- Cross-reference multiple sources to verify company information
- If a company seems perfect but you can't verify key details, note this in your thoughts and decide whether to include them

# OUTPUT FORMAT

When you have completed your research, output your findings in the following structured text format for each company:

---COMPANY---
NAME: [Company Name]
EMPLOYEE_COUNT: [Number of employees, or null if unknown]
ESTIMATED_REVENUE: [Revenue estimate like "$10M-$50M ARR", or null if unknown]
AREA_OF_EXPERTISE: [Primary business focus]
TECH_STACK: [Comma-separated list of technologies]
URL: [Company website]
LINKEDIN: [LinkedIn company page URL]
WHY_GOOD_FIT: [2-3 sentence explanation of why they're a good prospect]
DATA_SOURCES: [Comma-separated list of URLs where you found this information]
---END---

Example:
---COMPANY---
NAME: Acme Analytics
EMPLOYEE_COUNT: 150
ESTIMATED_REVENUE: $20M-$30M ARR
AREA_OF_EXPERTISE: B2B SaaS analytics platform for e-commerce
TECH_STACK: Python, React, PostgreSQL, AWS, Segment
URL: https://acmeanalytics.com
LINKEDIN: https://linkedin.com/company/acme-analytics
WHY_GOOD_FIT: Mid-size analytics company likely dealing with data pipeline challenges. Their job postings mention scaling issues with data infrastructure, which directly aligns with our data orchestration product.
DATA_SOURCES: https://acmeanalytics.com/careers, https://linkedin.com/company/acme-analytics, https://builtwith.com/acmeanalytics.com
---END---

Remember: You are the gatekeeper. Only companies that genuinely fit the offering should make it through your filter."""

In [ ]:


# ============================================================================
# CONTACT BASE MODEL
# ============================================================================
class Contacts(BaseModel):
   contacts: List["Contact"] = Field(description="List of qualified contacts")
   
class Contact(BaseModel):
    full_name: str = Field(description="Contact's full name")
    title: str = Field(description="Job title/role")
    department: Optional[str] = Field(description="Department (Sales, Engineering, etc.)")
    linkedin_url: str = Field(description="LinkedIn profile URL")
    email: Optional[str] = Field(description="Email address if found")
    phone: Optional[str] = Field(description="Phone number if found")
    why_relevant: str = Field(description="Why this person is a relevant contact for the product offering")
    company_name: str = Field(description="Company this contact works for")
    data_sources: List[str] = Field(default_factory=list, description="URLs where information was found")


# ============================================================================
# COMPANY TO STRING CONVERTER
# ============================================================================

def company_to_llm_string(company: Company) -> str:
    """Converts a Company object into a readable string representation for LLM processing."""
    
    tech_stack_str = ", ".join(company.tech_stack) if company.tech_stack else "Unknown"
    data_sources_str = "\n  - ".join(company.data_sources) if company.data_sources else "None provided"
    
    return f"""
COMPANY PROFILE
===============

Company Name: {company.name}
Website: {company.url}
LinkedIn: {company.linkedin}

SIZE & SCOPE
------------
Employees: {company.employee_count if company.employee_count else "Unknown"}
Estimated Revenue: {company.estimated_revenue if company.estimated_revenue else "Unknown"}
Area of Expertise: {company.area_of_expertise}

TECHNOLOGY
----------
Current Tech Stack: {tech_stack_str}

SALES CONTEXT
-------------
Why Good Fit: {company.why_good_fit}

RESEARCH SOURCES
----------------
  - {data_sources_str}
""".strip()


# ============================================================================
# CONTACT FINDER SYSTEM PROMPT
# ============================================================================

CONTACT_FINDER_SYSTEM = """You are a specialized research agent focused on identifying key decision-makers and relevant contacts at a specific company.

# YOUR MISSION
You will be provided with:
1. A company profile (name, size, industry, tech stack)
2. A product offering we want to sell to this company

Your job is to find 5-10 relevant contacts at this company who would be involved in evaluating, purchasing, or implementing this type of product.

# WHO TO FIND

Focus on decision-makers and influencers who would be involved in the buying process:

**Primary Targets (High Priority):**
- C-Suite executives (CEO, CTO, CIO, CDO, etc.) at smaller companies (<200 employees)
- VPs and Directors (Engineering, Product, Operations, IT, Data, etc.)
- Heads of relevant departments

**Secondary Targets (Medium Priority):**
- Senior managers in relevant teams
- Technical leads or architects who influence tool selection
- Product managers overseeing relevant areas

**Avoid:**
- Junior individual contributors (unless at very small companies)
- Recruiters or HR (unless selling HR product)
- Interns or entry-level roles

# RESEARCH METHODOLOGY

## Phase 1: Strategic Planning (Use Thought Management)
Before searching, use thought management tools to plan:
- Use `add_thought` to identify: What roles/titles would be involved in buying this product?
- Document your search strategy: Where will you look? What keywords will you use?
- Create a checklist of information to gather for each contact

## Phase 2: Broad Contact Discovery
Execute searches to find potential contacts:
- Search LinkedIn for employees with relevant titles
- Look at the company's "About" or "Team" pages
- Check recent press releases or news for quoted executives
- Review conference speaker lists, podcast appearances, webinars
- Look at GitHub contributors (for technical products)
- Use `add_thought` to track promising contacts and patterns

## Phase 3: Contact Qualification (Multi-Pass)

**First Pass - Identify Candidates:**
- Compile a list of 15-20 potential contacts with relevant titles
- Verify they currently work at the company (check for recent activity)
- Assess their likely influence in the buying decision

**Second Pass - Gather Details:**
- Find their LinkedIn profile URL (MANDATORY)
- Determine their exact title and department
- Look for email addresses (company website, GitHub, social profiles, conference materials)
- Look for phone numbers if publicly available
- Understand their role and why they're relevant to the product

**Third Pass - Prioritization:**
- Select the 5-10 BEST contacts (quality over quantity)
- Ensure diversity of roles (don't just find 10 engineers)
- Use `review_thoughts` to confirm you have the right mix

## Phase 4: Documentation & Finalization
- Use `resolve_thoughts` to mark each contact as fully researched
- Ensure you have 5-10 high-quality contacts
- Double-check that each contact has a LinkedIn URL (MANDATORY)

# CRITICAL REQUIREMENTS

1. **Always use thought management tools BEFORE searching**
   - Plan your approach before executing searches
   - Regularly check in with your thought list to track progress

2. **LinkedIn URLs are MANDATORY**
   - Every contact MUST have a LinkedIn profile URL
   - This is the most critical piece of information

3. **Focus on decision-makers**
   - Prioritize people who can actually say "yes" to a purchase
   - Consider the company size when determining seniority level

4. **5-10 contacts per company**
   - Better to have 6 excellent contacts than 10 mediocre ones
   - Aim for a mix of roles and seniority levels

5. **Explain relevance**
   - For each contact, clearly articulate why they're relevant
   - Connect their role to the product offering

# SEARCH STRATEGY TIPS

- Start with LinkedIn: "site:linkedin.com/in [Company Name] [Title]"
- Check company website team/about pages
- Look for recent company news mentioning specific people
- Search for the company name + role on Google
- Check if the company has an engineering blog with author bios
- Look at recent job postings - hiring managers are often listed
- For technical products, check GitHub for company organization members

# OUTPUT FORMAT

When you have completed your research, output your findings in the following structured text format for each contact:

---CONTACT---
FULL_NAME: [First Last]
TITLE: [Job Title]
DEPARTMENT: [Department name, or null if unknown]
LINKEDIN_URL: [LinkedIn profile URL]
EMAIL: [Email address, or null if not found]
PHONE: [Phone number, or null if not found]
WHY_RELEVANT: [2-3 sentence explanation of why this person is a relevant contact]
COMPANY_NAME: [Company name they work for]
DATA_SOURCES: [Comma-separated list of URLs where you found this information]
---END---

Example:
---CONTACT---
FULL_NAME: Sarah Johnson
TITLE: VP of Engineering
DEPARTMENT: Engineering
LINKEDIN_URL: https://linkedin.com/in/sarah-johnson-tech
EMAIL: sarah.johnson@acmeanalytics.com
PHONE: null
WHY_RELEVANT: As VP of Engineering at a mid-size analytics company, Sarah would be the primary decision-maker for data infrastructure tools. Her team is scaling and likely experiencing the exact pain points our product solves.
COMPANY_NAME: Acme Analytics
DATA_SOURCES: https://linkedin.com/in/sarah-johnson-tech, https://acmeanalytics.com/team
---END---

Remember: These contacts will be used for outreach. Only include people who are genuinely relevant and whose information you are confident about."""


# ============================================================================
# CONTACT PARSER SYSTEM PROMPT
# ============================================================================

CONTACT_PARSER_SYSTEM = """You are a specialized parsing agent that consolidates contact research into structured data.

# YOUR MISSION
You will receive raw text output from multiple contact research agents, each containing information about contacts found at different companies. Your job is to:

1. Parse all the contact information from the raw text
2. Validate that each contact has the required fields (especially LinkedIn URL)
3. Remove any duplicates
4. Output the final consolidated list in the proper structured format

# PARSING INSTRUCTIONS

Each contact in the input will be formatted as:
---CONTACT---
FULL_NAME: [name]
TITLE: [title]
DEPARTMENT: [dept or null]
LINKEDIN_URL: [url]
EMAIL: [email or null]
PHONE: [phone or null]
WHY_RELEVANT: [explanation]
COMPANY_NAME: [company]
DATA_SOURCES: [urls]
---END---

# VALIDATION RULES

For each contact, ensure:
1. **FULL_NAME is present and valid** - Must have at least first and last name
2. **TITLE is present** - Cannot be null or empty
3. **LINKEDIN_URL is present and valid** - This is MANDATORY, must be a real LinkedIn URL
4. **COMPANY_NAME is present** - Must match one of the companies being researched
5. **WHY_RELEVANT is present** - Must have explanation

If a contact is missing any of these critical fields, EXCLUDE it from the final output.

# DEDUPLICATION

If you find the same person listed multiple times (same name + company):
- Keep the entry with the most complete information
- Merge data_sources from both entries
- Use the most detailed WHY_RELEVANT explanation


YOU MUST complete all phases before returning the final output. It should contain the format specified above with little extra text.
"""

In [33]:
LeadFinder = rt.agent_node(
   name="Lead Finder",
    system_message=LEAD_FINDER_SYSTEM,
    tool_nodes=[query_internet, *thought_manager],
    llm=rt.llm.OpenAILLM("gpt-4.1"),
)

LeadFinderParser = rt.agent_node(
    name="Lead Finder Parser",
    system_message="You are a meticulous data parser. Your job is to extract structured company information from an input and put it into a structured form.",
    output_schema=Companies,
    llm=rt.llm.OpenAILLM("gpt-4.1"),
)

ContactFinder = rt.agent_node(
    name="Contact Finder",
    system_message=CONTACT_FINDER_SYSTEM,
    tool_nodes=[query_internet, *thought_manager],
    llm=rt.llm.OpenAILLM("gpt-4.1"),
)

ContactFinderParser = rt.agent_node(
    name="Contact Finder Parser",
    system_message=CONTACT_PARSER_SYSTEM,
    output_schema=Contacts,
    llm=rt.llm.OpenAILLM("gpt-4.1"),
)





In [ ]:
import asyncio


thought_store: dict[str, dict[str, str | None]] = defaultdict(dict)

@rt.session(context={"current_phase": "Lead Finding", "thought_store": thought_store})
async def flow():
    result = await rt.call(LeadFinder, "An tool that allows canadian companies to sred their work with ease. it is designed for mid level companies who do not have to pay a consulting firm and would prefer to use some sort of saas platform instead.")

    compainies = await rt.call(LeadFinderParser, result.text)

    rt.context.put("current_phase", "Contact Searching")
    contacts: dict[str, list[Contact]] = {}
    for company in compainies.structured.companies:
        company_str = company_to_llm_string(company)
        contact_results = await rt.call(ContactFinder, f"Find relevant contacts at the following company for selling our product:\n\n{company_str}\n\nProduct Offering: An agentic framework aimed at devs to help them build agents with ease. It offers a fully customizable python API. Paying products include a vizualization and sharing tooling for to view your runs through history.")
        contacts_list = await rt.call(ContactFinderParser, contact_results.text)

        contacts[company.name] = contacts_list.structured.contacts

    return contacts

    




In [40]:
companies, _ = await flow()

[+6066.012s] RT.Validation: WARNING  - You have set max_tool_calls to None. The llm will be able to make unlimited tool calls.
[+6066.014s] RT          : INFO     - START CREATED Lead Finder
[+6068.285s] RT          : INFO     - Lead Finder CREATED add_thought
[+6068.285s] RT          : INFO     - add_thought DONE
[+6070.868s] RT          : INFO     - Lead Finder CREATED add_thought
[+6070.878s] RT          : INFO     - add_thought DONE
[+6076.306s] RT          : INFO     - Lead Finder CREATED add_thought
[+6076.309s] RT          : INFO     - add_thought DONE
[+6080.012s] RT          : INFO     - Lead Finder CREATED add_thought
[+6080.015s] RT          : INFO     - add_thought DONE
[+6083.276s] RT          : INFO     - Lead Finder CREATED resolve_thoughts
[+6083.276s] RT          : INFO     - resolve_thoughts DONE
[+6087.291s] RT          : INFO     - Lead Finder CREATED resolve_thoughts
[+6087.302s] RT          : INFO     - resolve_thoughts DONE
[+6089.888s] RT          : INFO     - L

10


[+6193.765s] RT.Validation: WARNING  - You have set max_tool_calls to None. The llm will be able to make unlimited tool calls.
[+6193.767s] RT          : INFO     - START CREATED Contact Finder
[+6200.761s] RT          : INFO     - Contact Finder CREATED add_thought
[+6200.765s] RT          : INFO     - Contact Finder CREATED add_thought
[+6200.767s] RT          : INFO     - Contact Finder CREATED add_thought
[+6200.767s] RT          : INFO     - add_thought DONE
[+6200.775s] RT          : INFO     - add_thought DONE
[+6200.778s] RT          : INFO     - add_thought DONE
[+6202.875s] RT          : INFO     - Contact Finder CREATED query_internet
[+6202.875s] RT          : INFO     - Contact Finder CREATED query_internet
[+6202.887s] RT          : INFO     - Contact Finder CREATED query_internet
[+6202.892s] RT          : INFO     - Contact Finder CREATED query_internet
[+6202.894s] RT          : INFO     - Contact Finder CREATED query_internet
[+6202.902s] RT          : INFO     - Cont

In [46]:
for company_name, contact_list in companies.items():
    print(f"Company: {company_name} - Contacts Found: {','.join([f'{x.full_name}, {x.title}' for x in contact_list])}")

Company: CaseWare International - Contacts Found: Sam High, Chief Technology Officer (CTO),Danielle Supkis Cheek, Vice President, Head of Analytics and AI,Ronald Lau, Group Product Manager,D Kartik Narayanan, Senior Product Owner,Craig Thorvardson, Senior Software Development Manager,Matt Swann, Product Manager,Ricardo Navarrete, AI Platform Lead,Nabiha H, CPA, CA, Product Owner
Company: Deep Genomics - Contacts Found: Brendan Frey, Founder & Chief Innovation Officer,Asha Rostamianfar, Head of Engineering,Shawdee Eshghi, Vice President, Platform Biology,Leen Schafer, PhD, Director, Platform Engineering & AI,Stephanie Meler, Product,Albi Celaj, ML/AI Lead,Greg Hoffman, Chief Scientific Officer,Jean M Macklaim, PhD, Product Manager & Scientist,Varun Lodaya, Machine Learning Engineer II,Jill Vandenbosch, Head of AI Product
Company: FREDsense Technologies - Contacts Found: David Lloyd, CEO and Co-Founder,Emily Hicks, Co-Founder and Chief Operating Officer,Margaret (Maggie) Renaud-Young, Le

impo